In [185]:
import numpy as np
import pandas as pd

importance_matrix = pd.read_excel("../data/feature_importance.xlsx", sheet_name=4)
N = importance_matrix.shape[0]

Judgment_matrix_A = importance_matrix.drop("Unnamed: 0", axis=1).values
importance_matrix.iloc[:,0]

0     NAME_EDUCATION_TYPE
1             CODE_GENDER
2            FLAG_OWN_CAR
3         FLAG_OWN_REALTY
4        AMT_INCOME_TOTAL
5        NAME_INCOME_TYPE
6      NAME_FAMILY_STATUS
7                     age
8              occupation
9                 capital
10         hours-per-week
Name: Unnamed: 0, dtype: object

In [186]:
weights_W = []

for i in range(Judgment_matrix_A.shape[0]):
    weights_W.append(np.power(np.prod(Judgment_matrix_A[i]), 1/N))

std_weights_W = weights_W/np.sum(weights_W)
std_weights_W

array([0.06324513, 0.2670073 , 0.06735867, 0.01720726, 0.01998436,
       0.08033522, 0.18545356, 0.11098794, 0.05629297, 0.02864629,
       0.1034813 ])

In [187]:
np.sum(std_weights_W)

1.0

In [188]:
A_W = []
for i in range(Judgment_matrix_A.shape[0]):
    A_W.append(np.sum(std_weights_W*Judgment_matrix_A[i]))

lambda_max = np.sum(A_W / (N * std_weights_W))
lambda_max

12.379084394285133

In [189]:
CI = (lambda_max - N) / (N - 1)
RI = [0, 0, 0.68, 0.90, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49, 1.51, 1.54, 1.56]
CR = CI/RI[10]
CR

0.0913300923367638

In [190]:
feature_weights = pd.DataFrame({"Feature_name": importance_matrix.iloc[:,0], "Weight": std_weights_W})
feature_weights

,Feature_name,Weight
0,NAME_EDUCATION_TYPE,0.063245
1,CODE_GENDER,0.267007
2,FLAG_OWN_CAR,0.067359
3,FLAG_OWN_REALTY,0.017207
4,AMT_INCOME_TOTAL,0.019984
5,NAME_INCOME_TYPE,0.080335
6,NAME_FAMILY_STATUS,0.185454
7,age,0.110988
8,occupation,0.056293
9,capital,0.028646


In [191]:
feature_weights.to_csv('../data/feature_weights.csv', index=False)